In [12]:
from openai import OpenAI
import json
import os

GPT_MODEL_4 = "gpt-4-0125-preview"
OPEN_API_KEY = os.getenv("OPENAI_API_KEY")

client = OpenAI()
model = GPT_MODEL_4

def ask(prompt, client, model, temperature = 0):
    response = None
    
    response = client.chat.completions.create(
      model=model,
      messages=prompt,
      temperature=temperature,
    )

    return response.choices[0].message.content

def askJSON(prompt, client, model, temperature = 0):
    response = None
    
    response = client.chat.completions.create(
      model=model,
      messages=prompt,
      temperature=temperature,
      response_format={ "type": "json_object" },
    )

    return response.choices[0].message.content

In [13]:
def write_string_to_file(filename, content):
    try:
        with open(filename, 'a') as file:
            file.write(content)
        print(f"String has been written to {filename}")
    except IOError as e:
        print(f"An error occurred while writing to the file: {e}")

In [14]:
def read_file_to_list(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    return [line.strip() for line in lines]

In [15]:
import chardet

def read_file_content(file_path):
    # Detect the encoding
    with open(file_path, 'rb') as file:
        raw_data = file.read()
        result = chardet.detect(raw_data)
        encoding = result['encoding']
    
    # Read the file with the detected encoding
    try:
        with open(file_path, 'r', encoding=encoding) as file:
            content = file.read()
        return content
    except FileNotFoundError:
        return f"Error: The file at path {file_path} was not found."
    except UnicodeDecodeError:
        return f"Error: The file at path {file_path} cannot be decoded with the {encoding} encoding."
    except IOError:
        return f"Error: An I/O error occurred while reading the file at path {file_path}."


In [16]:
def write_json_to_txt_raw(json_data, file_name):
    with open(file_name, 'w') as file:
        for test_case, details in json_data.items():
            file.write(f"Test Case: {test_case}\n")
            for key, value in details.items():
                if isinstance(value, list):
                    file.write(f"{key}:\n")
                    for item in value:
                        file.write(f"  - {item}\n")
                else:
                    file.write(f"{key}: {value}\n")
            file.write("\n")

In [17]:
BASELINE_GENERATE_TC = """
read the test scenario's name and the corresponding use case specification to base on those information for generateing test steps for test cases and their following expected result.
Return the test cases in json format.
The JSON format should follow the following structure:
{
  "Test Case 1":{
    "testCaseName": "Clear name of the test case so tester know what to test when they first read",
    "objective": "Verify who doing what action or function in the test case and the summary of the final result of the test case",
    "testSteps": [
      "Step 1: Describe the step.",
      "Step 2: Describe the step.",
      "Step 3: Describe the step."
    ],
    "expectedResult": "You inform the tester what should they see after doing all the steps",
  },
}
If there are more than one test case for this scenario, continue writing other test case in this form."""

In [18]:
use_case_directory_path = r"D:\Dissertation-GPT\dataset\SpecificationData\Book(Github)"
save_path = r"D:\Dissertation-GPT\evaluate\baseline 2\book test case"

In [19]:
usecase_name_list = []
project_name = "Book"

In [20]:
for filename in os.listdir(use_case_directory_path):
    usecase_name_list.append (filename.split(".txt")[0])
print(usecase_name_list)

['Add a new author', 'Add a new book', 'Add a new genre', 'Display author details', 'Display book details', 'Display genre details', 'Display the list of authors', 'Display the list of books', 'Display the list of genres', 'Edit the author', 'Edit the book', 'Edit the genre', 'Remove the author', 'Remove the book', 'Remove the genre']


In [21]:
test_scenario_directory_path = r"D:\Dissertation-GPT\evaluate\baseline 2\book\ts"
save_testcase_path = r"D:\Dissertation-GPT\evaluate\baseline 2\book test case"

In [22]:

for usecase in usecase_name_list:
  base_name = os.path.basename(usecase)
  usecase_name =usecase
  usecase_path = os.path.join (use_case_directory_path,f"{usecase_name}.txt" )
  testscenario_path = os.path.join (test_scenario_directory_path,f"{usecase_name}.txt" )
  testscenario_list = read_file_to_list(testscenario_path)
  print(testscenario_path)
  print(testscenario_list)
  for testscenario in testscenario_list:
    content = "Test scenarios:"+ testscenario+  "\nUse case:" + read_file_content(usecase_path)
    print(content)
    promptExtractCondition = [
      { "role": "system", "content": BASELINE_GENERATE_TC},
      { "role": "user", "content": content}
    ]
    gpt_response = askJSON(promptExtractCondition, client, model)
    print(gpt_response)
    print()
    json_data = json.loads(gpt_response)
    testscenario = testscenario.replace('"','').replace(':','').replace('?','').replace('*','').replace('/','')
    testcase_path = os.path.join (save_testcase_path,f"{usecase_name}-{testscenario}.txt" )
    write_json_to_txt_raw(json_data,testcase_path )
  print("f{usecase_name} gen test case complete")

D:\Dissertation-GPT\evaluate\baseline 2\book\ts\Add a new author.txt
['1. VerifyClickCreateNewButton', '2. VerifyFormDisplayAfterClickingCreateNew', '3. VerifySubmitWithValidData', '4. VerifyErrorMessageForInvalidData', '5. VerifyDatabaseEntryOnSuccessfulSubmission', '6. VerifyErrorMessageOnDatabaseError', '7. VerifyAuthorListUpdateAfterNewAuthorAdded', '8. VerifyCancelOperationBeforeDataSubmission', '9. VerifyNoDatabaseEntryOnCancel', '10. VerifyNavigationAwayDoesNotSaveData', '11. VerifyMandatoryFieldsValidation', '12. VerifyMaxLengthValidationForNameField', '13. VerifyMaxLengthValidationForBioField', '14. VerifyInputTypeValidationForNameField', '15. VerifyTextAreaInputForBioField', '16. VerifyUniqueNameConstraint', '17. VerifySuccessfulSubmissionMessage', '18. VerifyFormResetAfterSubmission', '19. VerifyFormResetAfterCancellation', '20. VerifyNavigationToOtherSections', '21. VerifyAuthorRecordDeletionFunctionality (If applicable)', '22. VerifyAuthorRecordEditFunctionality (If applic

{
  "Test Case 1": {
    "testCaseName": "Verify successful creation of a new author record",
    "objective": "To verify that a user can successfully create a new author record by entering valid data and clicking the 'Submit' button",
    "testSteps": [
      "Step 1: Navigate to the author list page.",
      "Step 2: Click the 'Create new' button.",
      "Step 3: Enter valid data into the 'Name' and 'Bio' fields.",
      "Step 4: Click the 'Submit' button."
    ],
    "expectedResult": "A new author record is successfully added to the database, and the user is redirected to the list of authors with the newly added record visible."
  },
  "Test Case 2": {
    "testCaseName": "Verify incorrect data handling in new author creation",
    "objective": "To verify that the application displays error messages when incorrect data is entered in the author creation form",
    "testSteps": [
      "Step 1: Navigate to the author list page.",
      "Step 2: Click the 'Create new' button.",
     